# **Итоговый проект Deep Learning (семестр 1, осень 2025)**

**Студент**: Михаил Киселёв

**Stepik ID**: 1027756045

# Обзор литературы по Face Recognition: от классики до современных нейросетевых методов (2025)

## 1.1. Краткие выжимки из основополагающих статей
### 1.1.1 Eigenfaces (Turk & Pentland, 1991)
**Суть**: Первый успешный подход к распознаванию лиц с использованием метода главных компонент (PCA). Лицо представляется как линейная комбинация "собственных лиц" (eigenfaces) — основных компонент, извлеченных из набора обучающих изображений.

**Ключевая идея**: Проекция изображения лица в низкоразмерное пространство, где расстояние между проекциями (обычно евклидово) определяет сходство.

**Ограничения**: Чрезвычайно чувствителен к освещению, позе, выражению лица и требует строгого выравнивания.

### 1.1.2 Fisherfaces (Belhumeur et al., 1997)
**Суть**: Улучшение метода Eigenfaces с применением **линейного дискриминантного анализа (LDA)**, который максимизирует отношение межклассовой дисперсии к внутриклассовой.

**Ключевая идея**: В отличие от PCA, которая максимизирует общую дисперсию, LDA ищет такое подпространство, которое **лучше разделяет разные классы (лица)**, уменьшая влияние вариаций внутри одного класса (освещение, выражение).

**Результат**: Более высокая точность по сравнению с Eigenfaces в условиях изменяющегося освещения.

### 1.1.3 DeepFace (Taigman et al., 2014) — Начало эры глубокого обучения
**Суть**: Первая работа, показавшая, что глубокие сверточные нейросети (CNN) могут достигать точности, близкой к человеческой, на стандартном наборе данных **Labeled Faces in the Wild (LFW) (~97.35%)**.

**Ключевая инновация**: Использование сложного 3D-выравнивания лица ("frontalization") перед подачей в сеть, что значительно снижает вариативность позы.

**Значение**: Доказала практическую применимость глубокого обучения для FR и задала новый стандарт производительности.

### 1.1.4 FaceNet (Schroff et al., 2015) — Парадигма эмбеддингов
**Суть**: Предложена тройственная функция потерь (Triplet Loss) и архитектура для прямого обучения компактного евклидова векторного представления лица (128-D эмбеддинг).

**Ключевая идея**: Сеть обучается так, чтобы расстояние между эмбеддингами одного человека (anchor и positive) было меньше, чем расстояние между эмбеддингами разных людей (anchor и negative) на заданную величину (margin).

**Значение**: Установила новую парадигму **"одна сеть — один эмбеддинг"**, которая стала основой для большинства современных промышленных систем. Достигла 99.63% на LFW.

## 1.2. Основные современные алгоритмы и методы FR с использованием нейросетей (на 2025 год)
Современный пайплайн FR состоит из стандартных этапов:

- Детекция лица (например, RetinaFace, MTCNN, YOLO-based).

- Выравнивание и нормализация (ключевые точки, аффинные преобразования).

- Извлечение признаков (Feature Extraction) с помощью CNN.

- Сравнение эмбеддингов (метрика косинусного или евклидова расстояния).

### 1.2.1 Архитектуры сетей (Backbones)
Используются современные CNN и Vision Transformers, претренированные на огромных наборах данных (MS-Celeb-1M, WebFace, VGGFace2):

- **ResNet и его вариации (ResNet-50, ResNet-100, SE-ResNet)**: Стандартный выбор благодаря остаточным связям, решающим проблему затухания градиента.

- **EfficientNet**: Обеспечивает лучшее соотношение точности и вычислительной эффективности за счет масштабирования глубины, ширины и разрешения.

- **Vision Transformers (ViT, Swin Transformer)**: Набирают популярность, показывая SOTA-результаты на некоторых бенчмарках за счет глобального внимания к патчам изображения. Требуют больше данных для обучения.

### 1.2.2 Ключевые функции потерь (Loss Functions)
Эволюция loss-функций — основной драйвер прогресса в FR. Их цель — сделать **эмбеддинги одного класса максимально компактными, а разных классов — максимально разделенными в пространстве признаков**.

#### 1.1.2.1. Кросс-энтропийная потеря (Softmax Loss / CE Loss)

**Описание**: Классическая функция для классификации. Сеть обучается предсказывать конкретный класс (человека) из фиксированного набора.

**Проблема**: Напрямую не оптимизирует компактность внутри класса и разделимость между классами. Эмбеддинги одного человека могут быть разбросаны, если они правильно классифицированы.

#### 1.1.2.2. Loss-функции с добавлением маржи (Margin-based Losses)
Для решения проблемы Softmax Loss были предложены функции, которые добавляют угловую или косинусную маржу (margin) в пространстве признаков, создавая более жесткую границу решения.

- **ArcFace** (Additive Angular Margin Loss, 2019) — Наиболее популярная и эффективная.

  - **Формула**: $
\mathcal{L} = -\frac{1}{M}\sum_{i=1}^{M} \log\frac{e^{s*\cos(\theta_{y_i} + m)}}
{e^{s*\cos(\theta_{y_i} + m)} + \sum_{j \neq y_i} e^{s*\cos\theta_j}}
$

  - **Идея**: Добавляет аддитивную угловую маржу m (например, 0.5 радиан) непосредственно к углу θ между вектором весов класса W_yi и признаком x_i. Это сужает угловой сектор, отведенный для каждого класса.

  - **Преимущества**:

    - Четкая геометрическая интерпретация в угловом пространстве.

    - Обеспечивает максимально возможную разделимость между классами.

    - Превосходит другие методы на большинстве бенчмарков.

- **Другие важные функции с маржой**:

  - **CosFace (Large Margin Cosine Loss, 2018)**: Добавляет маржу к косинусу угла: $cos(θ) - m$.

  - **SphereFace (Angular Softmax, 2017)**: Пионерская работа, вводящая мультипликативную угловую маржу $(m*cos(θ))$.

### 1.2.3 Методы для работы с ранее не встречающимися классами (Open-Set Recognition)
Основная задача — сравнивать эмбеддинги лиц, которых не было в обучающей выборке.

- **Метрическое обучение (Metric Learning)**: Использование функций потерь (Triplet Loss, ArcFace), которые напрямую обучают "хорошему" пространству для сравнения.

- **Нормализация**: Все современные методы используют L2-нормализацию как эмбеддингов, так и весов последнего слоя. Это проецирует точки на гиперсферу, где сравнение косинусного подобия $(cos(θ) = (x·W) / (||x||*||W||))$ становится эквивалентным скалярному произведению нормализованных векторов.

- **Пороговая классификация**: Решение "свой/чужой" принимается путем сравнения косинусного сходства двух эмбеддингов с заранее выбранным порогом (например, 0.3-0.5).

## 1.3. Ссылки на ключевые статьи по нейросетевым алгоритмам FR
**ArcFace**: Deng, J., Guo, J., Xue, N., & Zafeiriou, S. (2019). ArcFace: Additive Angular Margin Loss for Deep Face Recognition. CVPR. https://arxiv.org/abs/1801.07698

**CosFace**: Wang, H., et al. (2018). CosFace: Large Margin Cosine Loss for Deep Face Recognition. CVPR. https://arxiv.org/abs/1801.09414

**SphereFace**: Liu, W., et al. (2017). SphereFace: Deep Hypersphere Embedding for Face Recognition. CVPR. https://arxiv.org/abs/1704.08063

**FaceNet**: Schroff, F., Kalenichenko, D., & Philbin, J. (2015). FaceNet: A Unified Embedding for Face Recognition and Clustering. CVPR. https://arxiv.org/abs/1503.03832

**VGGFace2**: Cao, Q., Shen, L., Xie, W., Parkhi, O. M., & Zisserman, A. (2018). VGGFace2: A dataset for recognising faces across pose and age. FG. https://arxiv.org/abs/1710.08092

**Более современные обзоры**: Deng, J., & Guo, J. (2020). A Survey of Face Recognition. https://arxiv.org/abs/2004.11823

## 1.4. FaceNet: Глубокое понимание
### 1.4.1. Концептуальный прорыв
До FaceNet большинство нейросетевых подходов (вроде DeepFace) использовали архитектуру, где последний слой был классификатором на N человек (где N — размер обучающей выборки). Сеть отвечала на вопрос: **"Кто это из известных мне людей?"**. Для сравнения двух лиц нужно было либо извлекать признаки из промежуточных слоев (что было неоптимально), либо запускать два лица через сеть и как-то сравнивать их активации.

**FaceNet предложил радикально иную идею**: обучить сеть не классификации, а прямому преобразованию лица в компактное, метрическое пространство.

### 1.4.2. Ключевые компоненты FaceNet
#### 1.4.2.1. Тройственная функция потерь (Triplet Loss).
Это сердце метода. Цель — научиться таким эмбеддингам, где:

- **Эмбеддинги одного человека** находятся близко друг к другу

- **Эмбеддинги разных людей** находятся далеко друг от друга

**Математическая формулировка**:
Для тройки изображений:

- **Anchor (A)** — базовое изображение человека

- **Positive (P)** — другое изображение того же человека

- **Negative (N)** — изображение другого человека

Сеть обучается минимизировать **Triplet Loss**:

$L = max( ||f(A) - f(P)||² - ||f(A) - f(N)||² + α, 0 )$

где:

$f(x)$ — эмбеддинг, полученный из сети (L2-нормализованный)

$||·||$ — евклидово расстояние

$α$ — **margin** (зазор), гиперпараметр, определяющий, насколько расстояние до negative должно быть больше, чем до positive

**Геометрическая интерпретация**: Loss стремится к нулю, когда расстояние между A и P на α меньше, чем расстояние между A и N. То есть positive должен находиться внутри сферы радиуса d(A,P), а negative — за пределами сферы радиуса d(A,P) + α.

#### 1.4.2.2. Выбор "тяжелых" троек (Triplet Mining)
Просто случайный выбор троек неэффективен — большинство из них будут тривиальными (negative и так далеко). **Ключевая инновация FaceNet — стратегия выбора троек**:

- **Semi-hard negative mining**: Выбираются такие тройки, где Negative находится дальше Positive от Anchor, но разница меньше margin α:
d(A, P) < d(A, N) < d(A, P) + α

- **Hard negative mining**: Еще более жесткий отбор, где Negative ближе к Anchor, чем Positive: d(A, N) < d(A, P)

Это обеспечивает **информативные примеры**, которые действительно "заставляют" сеть учиться разделять похожие, но разные лица.

### 1.4.3. Архитектура сети
FaceNet использовала **GoogleNet (Inception v1)** в качестве backbone-сети, но концепция была архитектурно-независимой. Главное — последние слои:

- Обычные сверточные слои для извлечения признаков

- Слой глобального среднего пулинга (Global Average Pooling) вместо полносвязных слоев

- L2-нормализация выхода, чтобы все эмбеддинги лежали на гиперсфере единичного радиуса

- Выход — 128-мерный вещественный вектор (эмбеддинг)

**Почему именно 128 измерений?**
Это эмпирический выбор авторов, представляющий баланс между:

- Достаточной емкостью для кодирования информации о лице

- Компактностью для быстрого сравнения и хранения

- Устойчивостью к переобучению

### 1.4.4. Парадигма "Одна сеть — один эмбеддинг"
**Раньше (DeepFace-подход)**:

[Изображение лица] → [Сеть-классификатор] → [Вероятности по N классам]

Для сравнения двух лиц нужно сравнивать векторы вероятностей или признаки из промежуточных слоев.

**Теперь (FaceNet-подход)**:

[Изображение лица A] → [Сеть-эмбеддинг] → [Вектор $f(A) ∈ R^{128}$]

[Изображение лица B] → [Та же самая сеть] → [Вектор $f(B) ∈ R^{128}$]

Сравнение: $distance = ||f(A) - f(B)||$

**Преимущества парадигмы**
1. **Масштабируемость**
- **Добавление нового человека** не требует переобучения всей сети

- Для хранения информации о новом лице нужно лишь сохранить **128 чисел (эмбеддинг)**

- Поиск среди миллиардов лиц сводится к задаче поиска ближайших соседей (k-NN) в 128-мерном пространстве

2. **Естественность для Open-Set Recognition**
- **Классификационный подход**: Работает только с известными классами. Что делать с новым человеком?

- **Эмбеддинговый подход**: Все лица (известные и неизвестные) проецируются в одно и то же пространство. Новый человек просто получает свои координаты в этом пространстве. Решение "свой/чужой" принимается по порогу расстояния.

3. **Универсальность применения**
Один и тот же эмбеддинг можно использовать для разных задач:

- Верификация (1:1): "Это тот же самый человек?" → сравнить 2 эмбеддинга

- Идентификация (1:N): "Кто этот человек?" → поиск ближайшего эмбеддинга в базе

- Кластеризация: Группировка лиц по сходству без учителя

- Поиск дубликатов в фотоархивах

4. **Интерпретируемость пространства**
Эмбеддинговое пространство FaceNet обладает интересными свойствами:

- Линейные операции: f(король) - f(мужчина) + f(женщина) ≈ f(королева) (аналогия word2vec, но для лиц)

- Непрерывность: Малые изменения в изображении → малые изменения в эмбеддинге

- Семантическая организация: Похожие лица (близнецы, родственники) находятся рядом

## 1.5. Эволюция после FaceNet
Хотя FaceNet представил парадигму эмбеддингов, у Triplet Loss были недостатки:

- Сложность выбора троек — вычислительно дорого

- Медленная сходимость — нужно много эпох

- Чувствительность к гиперпараметрам (особенно margin α)

Поэтому современные методы (ArcFace, CosFace) используют модифицированный Softmax с маржой, который:

- Сохраняет преимущества эмбеддингового подхода

- Обучается быстрее и стабильнее

- Дает еще более качественные эмбеддинги

## 1.6. Промышленное применение сегодня
Парадигма "одна сеть — один эмбеддинг" стала стандартом де-факто:

- Apple Face ID: Использует подобный подход (нейросеть + эмбеддинги)

- Системы видеонаблюдения: Хранение и поиск по эмбеддингам

- Социальные сети: Автотегирование фотографий

- Паспортный контроль: Быстрая верификация

## 1.7. Критические замечания
**Уязвимость к атакам**: Малые адверсарные возмущения могут изменить эмбеддинг. **Адверсарные возмущения** — это специально созданные, малозаметные для человека искажения входных данных (изображений, аудио, текста), которые заставляют нейронную сеть совершать предсказуемые ошибки. **Ключевая особенность**: эти искажения настолько малы, что человеческий глаз их практически не замечает, но для нейросети они кардинально меняют результат работы.

**Проблема смещения (bias)**: Качество разное для разных демографических групп

**Вычислительная стоимость инференса**: Нужно прогонять каждое лицо через глубокую сеть

**Проблема негативных сэмплов**: При огромных базах (миллионы лиц) "hard negative mining" становится вычислительно сложным

## 1.8. Заключение
**FaceNet** совершил переворот, показав, что:

- Можно обучать эмбеддинги напрямую, а не как побочный продукт классификации

- Евклидово/косинусное расстояние в эмбеддинговом пространстве — естественная и эффективная мера сходства лиц

- Парадигма "одна сеть для всех" масштабируется до миллиардов лиц

Эта парадигма настолько успешна, что пережила саму архитектуру FaceNet: современные SOTA-модели (с ArcFace и другими loss-функциями) все еще следуют тому же принципу — одна сеть преобразует лицо в вектор, а сравнение векторов дает ответ. **Это фундаментальный сдвиг в мышлении: от "распознавания" к "представлению"**.